In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt
from sklearn import tree
#import pydotplus
import collections
from IPython.display import Image  
#import pydotplus
from sklearn.tree import export_graphviz

In [104]:
import copy

In [105]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [118]:
df = pd.read_stata("Econ_484_data_final.dta")

In [107]:
# df.isin(["''"]).sum(axis=1).sum()

In [108]:
df = df.drop("DAYSWAIT_CHRON", axis=1)

In [109]:
df = df.dropna(axis=0, how='any')

In [110]:
df['year'] = pd.DatetimeIndex(df["TX_DATE"]).year
df['month'] = pd.DatetimeIndex(df["TX_DATE"]).month
df = df.drop("TX_DATE", axis=1)

### Making dummy variables

In [111]:
dummy_columns = ['gender', 'abo', 'EXH_PERIT_ACCESS', 'EXH_VASC_ACCESS', 'PREV_TX', 'PREV_KI_TX',
                 'MALIG_TRR', 'txkid', 'ABO_DON', 'DON_TY', 'GENDER_DON',
                 'HOME_STATE_DON', 'ABO_MAT', 'GRF_STAT_KI', 'DWFG_KI', 'PREV_TX_ANY', 'PX_STAT',
                 'SHARE_TY', 'AGE_GROUP', 'malig', 'LT_ONE_WEEK_DON', 'RECOV_OUT_US', 'year', 'month',
                 'PERM_STATE']
funky_columns = ['TX_PROCEDUR_TY_KI']

In [112]:
df_non_dummy = df.drop(dummy_columns, axis=1)
#df_non_dummy

In [113]:
df_dummified = pd.get_dummies(df[dummy_columns])


In [114]:
df = df_non_dummy.merge(df_dummified, left_index=True, right_index=True)


In [119]:
df.groupby('PERM_STATE')['DAYSWAIT_CHRON_KI'].mean().sort_values(ascending=True)

PERM_STATE
AS     113.500000
UT     396.112258
OR     406.574068
KY     408.346516
NE     413.064571
AR     434.404139
ME     439.108321
ZZ     439.984223
ID     440.481081
KS     442.715304
IA     448.285899
NV     455.524530
FL     475.629359
WV     476.939924
VT     479.534815
NA     486.886744
OK     495.519208
ND     501.517791
NH     502.474780
WI     509.442720
SD     530.993789
MO     534.463941
IN     540.128632
MT     550.881002
SC     551.050478
OH     553.236842
TX     560.038818
AZ     561.054896
RI     580.219005
MN     595.614792
MD     601.100217
TN     605.647220
PA     610.655378
GA     616.430043
MS     618.979475
WY     624.157778
AK     630.578411
PR     631.389474
WA     633.527757
LA     635.195573
MI     636.849154
IL     636.974154
NM     640.207407
CO     644.463526
GU     648.111111
NJ     659.606037
VA     659.966429
DC     683.127622
CT     691.657632
MA     694.375915
DE     695.335235
NC     708.891229
NY     712.433919
VI     714.686869
AL     756.400739

In [38]:
df.groupby('region')['DAYSWAIT_CHRON_KI'].median().sort_values(ascending=True)

region
4     320.0
7     338.0
8     343.0
3     346.0
10    358.0
11    388.0
2     392.0
1     410.0
6     413.0
9     430.0
5     437.0
Name: DAYSWAIT_CHRON_KI, dtype: float64

In [44]:
df['quint']=df.groupby('PERM_STATE')['DAYSWAIT_CHRON_KI'].transform(pd.qcut,5,labels=False)


In [50]:
ew = df.pivot_table('DAYSWAIT_CHRON_KI', index='PERM_STATE', columns= 'quint', aggfunc='median')
ew.describe()

quint,0.0,1.0,2.0,3.0,4.0
count,59.000000,57.000000,58.000000,57.000000,59.000000
mean,59.135593,171.745614,389.844828,721.421053,1402.822034
std,87.225719,39.329324,139.510146,157.125740,361.966084
min,3.000000,54.000000,221.000000,455.000000,134.000000
25%,42.000000,146.000000,322.125000,589.000000,1146.750000
50%,48.000000,173.000000,378.750000,734.000000,1413.000000
75%,55.000000,189.000000,433.000000,846.000000,1595.500000
max,709.000000,301.000000,1234.000000,1174.500000,2185.000000


In [116]:
y_variable = 'DAYSWAIT_CHRON_KI'
X_variables = list(df.columns[df.columns != y_variable])
X = df[X_variables].to_numpy()
y = df[y_variable].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

forest = RandomForestRegressor(n_estimators=5)
model = forest.fit(X_train, y_train)

In [117]:
model.score(X_test,y_test)

0.833438273775404

In [69]:
df.query('PERM_STATE_CA==1')['quint'].value_counts()


0.0    9405
2.0    9365
4.0    9358
3.0    9354
1.0    9313
Name: quint, dtype: int64

In [81]:
num_CA_1 = X[226,:].copy().reshape(1,-1)
# Has long WL time
num_CA_2 = X[77,:].copy().reshape(1,-1)
num_CA_3 = X[224,:].copy().reshape(1,-1)

In [18]:
num_21  = pd.DataFrame(num_21, columns = df[X_variables].columns)
num_21

,NUM_PREV_TX,A1,A2,B1,B2,DR1,DR2,REM_CD,END_STAT,INIT_AGE,...,PERM_STATE_TX,PERM_STATE_UT,PERM_STATE_VA,PERM_STATE_VI,PERM_STATE_VT,PERM_STATE_WA,PERM_STATE_WI,PERM_STATE_WV,PERM_STATE_WY,PERM_STATE_ZZ
0,0.0,11.0,28.0,8.0,44.0,3.0,4.0,4.0,4010.0,43.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
model.predict(num_CA_1)



array([1810.])

In [91]:
model.predict(num_CA_2)

array([68.4])

In [92]:
model.predict(num_CA_3)

array([890.8])

In [95]:

#num_21 =  X[21,:172].copy().reshape(1, -1)
List = []
for i in range(58):
    A = np.zeros((58,), dtype=int)
    A[i]=1
    num_10000 = np.concatenate((X[224,:173].copy(),A), axis = None).reshape(1,-1)
    List.append(model.predict(num_10000))
    print(model.predict(num_10000))
    

[666.2]
[978.8]
[622.2]
[666.2]
[554.]
[890.8]
[723.4]
[666.2]
[760.8]
[666.2]
[486.4]
[833.2]
[666.2]
[1246.2]
[532.8]
[452.4]
[718.2]
[666.2]
[578.4]
[482.6]
[666.2]
[778.]
[733.6]
[666.2]
[826.2]
[727.8]
[666.2]
[666.2]
[666.2]
[703.6]
[641.2]
[917.6]
[666.2]
[571.4]
[666.2]
[853.2]
[667.]
[513.8]
[1019.]
[666.2]
[685.8]
[502.6]
[666.2]
[733.4]
[666.2]
[666.2]
[666.2]
[669.2]
[666.2]
[481.4]
[666.2]
[666.2]
[666.2]
[836.2]
[733.2]
[666.2]
[705.4]
[546.8]


In [86]:
state_names = ['AK','AL','AR','AS','AZ','CA','CO','CT','DC','DE','FL','GA',
               'GU','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME',
               'MI','MN','MO','MP','MS','MT','NA','NC','ND','NE','NH','NJ',
               'NM','NV','NY','OH','OK','OR','PA','PR','RI','SC','SD','TN',
               'TX','UT','VA','VI','VT','WA','WI','WV','WY','ZZ']


In [87]:
# Person 1
List = np.asarray(List)
#List.reshape(-1,1)

wl_times = pd.DataFrame()
wl_times['state']= state_names
wl_times['226']= List


In [94]:
# Person 2
List = np.asarray(List)
wl_times['77']=List


In [96]:
# Person 3
List = np.asarray(List)
wl_times['224']=List

In [43]:
# Person 4
List = np.asarray(List)
wl_times['173781']=List

In [45]:
# Person 5
List = np.asarray(List)
wl_times['311525']=List

In [47]:
# Person 6
List = np.asarray(List)
wl_times['336194']=List

In [49]:
# Person 7
List = np.asarray(List)
wl_times['298122']=List

In [51]:
# Person 8
List = np.asarray(List)
wl_times['343924']=List

In [53]:
# Person 9
List = np.asarray(List)
wl_times['22757']=List

In [101]:
wl_times

,state,226,77,224
0,AK,1831.6,56.4,666.2
1,AL,1831.6,38.0,978.8
2,AR,1831.6,32.2,622.2
3,AS,1831.6,27.8,666.2
4,AZ,1831.6,34.0,554.0
5,CA,1810.0,68.4,890.8
6,CO,1831.6,27.8,723.4
7,CT,1831.6,27.8,666.2
8,DC,1831.6,27.8,760.8
9,DE,1831.6,27.8,666.2


In [102]:
wl_times.describe()

,226,77,224
count,58.000000,58.000000,58.000000
mean,1831.717241,44.362069,690.158621
std,4.724558,55.599930,136.637647
min,1810.000000,16.400000,452.400000
25%,1831.600000,27.800000,666.200000
50%,1831.600000,27.800000,666.200000
75%,1831.600000,46.150000,726.700000
max,1860.000000,437.200000,1246.200000


In [56]:
WA = df.query('PERM_STATE_WA == 1')


In [57]:
HI = df.query('PERM_STATE_HI == 1')

In [58]:
KY= df.query('PERM_STATE_KY==1')

In [59]:
IA = df.query('PERM_STATE_IA==1')

In [97]:
y_variable = 'DAYSWAIT_CHRON_KI'
X_variables = list(CA.columns[CA.columns != y_variable])
X = CA[X_variables].to_numpy()
y = CA[y_variable].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

forest = RandomForestRegressor(n_estimators=5)
model_CA = forest.fit(X_train, y_train)

In [98]:
model_CA.predict(num_CA_1)

array([1946.])

In [99]:
model_CA.predict(num_CA_2)

array([61.2])

In [100]:
model_CA.predict(num_CA_3)

array([914.])